# Inicialización

In [1]:
from pyspark import SparkContext
sc=SparkContext()

In [2]:
rdd_derecho=sc.textFile('cociente_clean_uniform_od.csv').map(lambda x: x.split(';'))
rdd_derecho=rdd_derecho.map(lambda x: [x[0], x[1],x[2],x[3],eval(x[4])])
print('Número de registros: ', rdd_derecho.count())

Número de registros:  32152


In [3]:
rdd_izquierdo=sc.textFile('cociente_clean_uniform_oi.csv').map(lambda x: x.split(';'))
rdd_izquierdo=rdd_izquierdo.map(lambda x: [x[0], x[1],x[2],x[3],eval(x[4])])
print('Número de registros: ', rdd_izquierdo.count())

Número de registros:  34782


# Rutinas necesarias

In [5]:
from fuzzywuzzy import fuzz

In [6]:
def match_complejo(x,y,npiv):
    if npiv==0:
        if x[2]==y[2]:
            if x[1]==y[1] or x[0]==y[0]:
                return True
            else:
                if fuzz.ratio(x[1],y[1])>90:
                    return True
                if fuzz.ratio(x[0],y[0])>90:
                    return True
            return False
        else:
            if fuzz.ratio(x[0],y[0])>90:
                if fuzz.ratio(x[1],y[1])>90:
                    return True
            return False
    if npiv==1 or npiv==2:
        if x[0]==y[0]:
            if x[2]==y[2]:
                return True
            if fuzz.ratio(x[1],y[1])>90:
                return True
        elif x[2]==y[2]:
            if fuzz.ratio(x[1],y[1])>90:
                return True
        return False
    if npiv==3:
        if x[0]==y[0]:
            if x[1]==y[1] or x[2]==y[2]:
                return True
            else:
                if fuzz.ratio(x[1],y[1])>90:
                    return True
                if fuzz.ratio(x[2],y[2])>90:
                    return True
            return False
        else:
            if fuzz.ratio(x[2],y[2])>90:
                if fuzz.ratio(x[1],y[1])>90:
                    return True
            return False

In [7]:
def clave_valor(x,npiv):
    clave=x[npiv]
    del x[npiv]
    return [clave,x]

def clave_valor_inversa(x,npiv):
    x[1].insert(npiv,x[0])
    return x[1]

In [8]:
def compare(x,npiv,funcion_matching):
    if len(x[1])==1:                          
        return (x[0],list(x[1]))
    else:                                      
        lista=list(x[1])
        for i in reversed(range(len(lista))):            
            for j in range(0,i):
                if funcion_matching(lista[i],lista[j],npiv):
                    lista=merge(lista,i,j)
                    break
        return (x[0],lista) 

In [9]:
def merge(lista,i,j):
    if len(lista[j][3])>len(lista[i][3]):
        lista[j][3]|=lista[i][3];
    elif len(lista[j][3])<len(lista[i][3]):
        lista[i][3]|=lista[j][3];
        lista[j]=lista[i];
    else:
        if lista[i][1]>=lista[j][1]:
            lista[j][1]=lista[i][1]
        if lista[i][0]>=lista[j][0]:
            lista[j][0]=lista[i][0]
        if lista[i][2]>=lista[j][2]:
            lista[j][1]=lista[i][1]
    
    del lista[i];
    return lista

In [10]:
def pivote_info(rdd,npiv,funcion_matching):
    nreg=rdd.count()
    t=time.time()
    
    rdd2=rdd.map(lambda x: clave_valor(x,npiv)).groupByKey()
    bloques=rdd2.count()
    comparaciones=rdd2.map(lambda x: len(list(x[1]))**2).sum()
    rdd2=rdd2.map(lambda x: compare(x,npiv,funcion_matching)).flatMapValues(lambda x: x).map(lambda x: clave_valor_inversa(x,npiv))
    
    tiempo=time.time()-t
    nfinal=rdd2.count()
    duplicados=nreg-nfinal
    info=[round(tiempo,2),duplicados,nfinal,bloques,round(nreg/bloques,2),comparaciones,round(duplicados**2/comparaciones)]
    return rdd2, info

# Ejecución 

In [11]:
import time
from pandas import DataFrame

In [12]:
rdd_derecho_final=rdd_derecho
errores=[0]*4
for i in [3,1,2,0]:
    rdd_derecho_final,info=pivote_info(rdd_derecho_final,i,match_complejo)
    print(info)
    errores[i]=info[1]    
print(sum(errores))

rdd_izquierdo_final=rdd_izquierdo
errores=[0]*4
for i in [3,1,2,0]:
    rdd_izquierdo_final,info=pivote_info(rdd_izquierdo_final,i,match_complejo)
    print(info)
    errores[i]=info[1]    
print(sum(errores))

[1.22, 5548, 26604, 15207, 2.11, 101838, 302]
[0.83, 3198, 23406, 18862, 1.41, 83810, 122]
[0.68, 98, 23308, 3652, 6.41, 2814546, 0]
[0.95, 1, 23307, 22129, 1.05, 932936, 0]
8845
[0.95, 6755, 28027, 15616, 2.23, 118832, 384]
[0.83, 4162, 23865, 19050, 1.47, 95639, 181]
[0.56, 173, 23692, 3699, 6.45, 2926167, 0]
[0.85, 2, 23690, 22333, 1.06, 1140502, 0]
11092


Obtenemos por este "camino" 8835 y 11089 errores, bastante bien frente a los detectados por enfoque bruto

Nota: al cambiar el fichero original de datos a utf8 y rehacer todo el procesado,
los números son ligeramente diferentes 8845 y 11092, en todo caso al alza.

In [14]:
def save_rdd_to_csv(rdd, file_name, n_muestras=1):
    with open(file_name, 'w') as f:
        for id,apellidos,nombre,fecha_nac,muestras in rdd.collect():
            if len(muestras) >= n_muestras:
                result = ';'.join([str(id),apellidos,nombre,fecha_nac]+[repr(muestras)])
                print(result, file=f)

In [15]:
save_rdd_to_csv(rdd_derecho_final, "merged_cociente_clean_uniform_od.csv")

# Postprocesado final

Hay menos de 1000 registros sin número de historia, podemos ver si los apellidos
y la fecha de nacimiento coinciden con algún otro campo. 

Estoy seguro de que podría integrarse en match complejo...

# Recuperación de los datos de las campimetrías

In [17]:
def recover_from_index(name, index_set):
    result = []
    with open(name) as input_file:
        for line in input_file:
            if int(line.split(';')[0]) in index_set:
                result.append(line)
    return result

La función recover_from_index recorre todo el archivo 'clean_uniform_od.csv' para encontrar los datos asociados a un índice, tarda muchísimo. En vez de eso, metemos el archivo en una rdd y operamos un join a la rdd libre de errores con los índices como clave, para luego quedarnos con los datos ya limpios de cada campimetría.

In [ ]:
rdd_datos_derecho=sc.textFile('clean_uniform_od.csv').\
                        map(lambda x: (int(x.split(';')[0]),x.split(';')[1:]))
rdd_juntada_derecho=rdd_derecho_final.flatMap(lambda x: [(i,x[:-1]) for i in x[-1]])
rdd_combinacion_derecho=rdd_datos_derecho.join(rdd_juntada_derecho).\
                        mapValues(lambda x: x[1]+x[0][4:]).\
                        map(lambda x: ';'.join(x[1]))

rdd_datos_izquierdo=sc.textFile('clean_uniform_oi.csv').\
                        map(lambda x: (int(x.split(';')[0]),x.split(';')[1:]))
rdd_juntada_izquierdo=rdd_izquierdo_final.flatMap(lambda x: [(i,x[:-1]) for i in x[-1]])
rdd_combinacion_izquierdo=rdd_datos_izquierdo.join(rdd_juntada_izquierdo).\
                        mapValues(lambda x: x[1]+x[0][4:]).\
                        map(lambda x: ';'.join(x[1]))

rdd_combinacion_derecho.take(5)


*Cuidado:* si los archivos ya existen, el comando saveAsTextFile da error 

In [19]:
with open('deduplicated_uniform_od.csv', 'w') as f:
        for line in rdd_combinacion_derecho.collect():
            print(line, file=f)

In [20]:
with open('deduplicated_uniform_oi.csv', 'w') as f:
        for line in rdd_combinacion_izquierdo.collect():
            print(line, file=f)